to know how many cases are in this file

In [1]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


get the title list

In [38]:
import fitz
import re

def extract_title(pdf_path, patterns):
# def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            # result_string = re.sub(r',\s*\d+\s*F\.\d+\s*\d*', '', span["text"])
                            # extracted_text_list.append(result_string.strip())
                            result_string = span["text"].strip()
                            # Try matching each pattern
                            for pattern in patterns:
                                result_string = re.sub(pattern, '', result_string)
                            extracted_text_list.append(result_string)    

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list

In [46]:
import fitz
import re

def extract_title(pdf_path):
# def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            pattern = r'(.*),(?=\s*\d+\s*[A-Za-z. ]+\s*\d+$)'
                            match = re.match(pattern, span["text"])
                            if match:
                                extracted_text = match.group(1).strip()
                                print(extracted_text)
                                extracted_text_list.append(extracted_text)    

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list

In [51]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r',[^,]*$', '', span["text"].strip())
                            extracted_text_list.append(result_string)

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list


In [52]:
pdf_file_path = "D:/antitrust/verdict/Federal Circuit (30).PDF"
# pdf_file_path = "D:/antitrust/verdict/Federal District (30).PDF"
# patterns = [
#     r',\s*\d+\s*F\.\d+\s*\d*',
#     r',\s*\d+\s*[A-Za-z. ]+\s*\d+$'
# ]
# result_list = extract_title(pdf_file_path,patterns)
result_list = extract_title(pdf_file_path)

In [53]:
result_list

['Paterson Parchment Paper Co. v. Story Parchment Co.',
 'Union Leader Corp. v. Newspapers of New England, Inc.',
 'Convertible Top Replacement Co. v. Aro Mfg. Co.',
 'International Shoe Machine Corp. v. United Shoe Machinery Corp.',
 'Haverhill Gazette Co. v. Union Leader Corp.',
 'Quinn v. Mobil Oil Co.',
 'Farmington Dowel Prods. Co. v. Forster Mfg. Co.',
 'George R. Whitten, Jr., Inc. v. Paddock Pool Builders, Inc.',
 'Dickstein v. Du Pont',
 'Carroll v. Protection Maritime Ins. Co.',
 'Barry v. St. Paul Fire & Marine Ins. Co.',
 'Engine Specialties, Inc. v. Bombardier, Ltd.',
 'A.D.M. Corp. v. Sigma Instruments, Inc.',
 "Allied Int'l v. International Longshoremen's Ass'n",
 "B. C. Recreational Indus. v. First Nat'l Bank",
 'Cordova & Simonpietri Ins. Agency, Inc. v. Chase Manhattan Bank N.A.',
 'Auburn News Co. v. Providence Journal Co.',
 'Gilbuilt Homes, Inc. v. Continental Homes of New England, etc.',
 '9 to 5 Organization for Women Office Workers v. Board of Governors of Feder

In [54]:
import csv

get other information and write them into csv

In [57]:
import fitz

def getAndWrite(pdf_path, case_list, output_csv):

    doc = fitz.open(pdf_path)
    matching_texts = []
    # date = ''
    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = None
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                for case_name in case_list:
                                    if span["text"].strip() == case_name.strip():
                                        title = span["text"].strip()
                                        # print(title)
                                        # Set the flag to consider 'number': 2 and 'number': 3 text
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
                                # print(court)
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                # date = span["text"].strip()
                                # date += span["text"].strip()
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                # Stop considering 'number': 2, 'number': 3, and 'number': 4 text
                                consider_number_2 = False
                                print({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page':page_num+1 })
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page':page_num+1 })
    
    doc.close()
    
    # Save matching_texts to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)



In [17]:
pdf_file_path = "D:/antitrust/verdict/State Surperior Court (CA) (30).PDF"
# pdf_file_path = "D:/antitrust/verdict/Federal District (30).PDF"
result_list = extract_title(pdf_file_path)

In [59]:
result_texts = getAndWrite(pdf_file_path, result_list, 'Federal Circuit (30).csv')


{'Title': 'Paterson Parchment Paper Co. v. Story Parchment Co.', 'Court': 'Circuit Court of Appeals, First Circuit', 'Date': 'January 23, 1930', 'No': 'No. 2372', 'page': 7}
{'Title': 'Union Leader Corp. v. Newspapers of New England, Inc.', 'Court': 'United States Court of Appeals for the First Circuit', 'Date': 'September 22, 1960, Heard ; December 2, 1960, Decided', 'No': 'Nos. 5619, 5620', 'page': 16}
{'Title': 'Convertible Top Replacement Co. v. Aro Mfg. Co.', 'Court': 'United States Court of Appeals for the First Circuit', 'Date': 'December 13, 1962', 'No': 'Nos. 5991, 5993', 'page': 24}
{'Title': 'International Shoe Machine Corp. v. United Shoe Machinery Corp.', 'Court': 'United States Court of Appeals for the First Circuit', 'Date': 'March 11, 1963', 'No': 'No. 6043', 'page': 31}
{'Title': 'Haverhill Gazette Co. v. Union Leader Corp.', 'Court': 'United States Court of Appeals for the First Circuit', 'Date': 'July 6, 1964', 'No': 'Nos. 6175, 6191', 'page': 44}
{'Title': 'Quinn v.